<a href="https://colab.research.google.com/github/thangarani/project/blob/master/text_summarization_pegasus.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Pegasus Text Summarization - Extractive

---



From [pegasus](https://github.com/google-research/pegasus) project.

First, change to GPU runtime

MENU > Runtime > Change runtime type > Hardware accelerator = GPU 

In [ ]:
%tensorflow_version 1.x

TensorFlow 1.x selected.


In [ ]:
# import tensorflow as tf
# tf.__version__

In [ ]:
!npx degit google-research/pegasus -f

npx: installed 1 in 2.764s
> destination directory is not empty. Using --force, continuing
> cloned google-research/pegasus#master


## requirements.txt

In [ ]:
# absl-py
# mock
# numpy
# rouge-score
# sacrebleu
# sentencepiece
# tensorflow-text==1.15.0rc0
# tensor2tensor==1.15.0
# tfds-nightly
# tensorflow-gpu==1.15.2

In [ ]:
# check and do it manually
!pip install mock rouge-score sacrebleu sentencepiece tensorflow-text==1.15.1
# the rest = installed already

     |████████████████████████████████| 61kB 5.0MB/s 
     |████████████████████████████████| 1.1MB 18.5MB/s 
     |████████████████████████████████| 9.1MB 34.0MB/s 
     |████████████████████████████████| 17.1MB 196kB/s 
  Created wheel for gast: filename=gast-0.2.2-cp36-none-any.whl size=7540 sha256=8a514a3b512a9c9ac4c77cc9868a60a15ad828e5cd6e9b4882dbbee86c527ce7
  Stored in directory: /root/.cache/pip/wheels/5c/2e/7e/a1d4d4fcebe6c381f378ce7743a3ced3699feb89bcfbdadadd
Successfully built gast
  Found existing installation: gast 0.3.3
    Uninstalling gast-0.3.3:
      Successfully uninstalled gast-0.3.3


You can restart or not. It's fine. Six isn't that important

## pretrained

In [ ]:
!mkdir ckpt
!gsutil cp -r gs://pegasus_ckpt/ ckpt/

Copying gs://pegasus_ckpt/c4.unigram.newline.10pct.96000.model...
Copying gs://pegasus_ckpt/c4.unigram.newline.10pct.96000.vocab...
Copying gs://pegasus_ckpt/checkpoint...
Copying gs://pegasus_ckpt/model.ckpt-1500000.data-00000-of-00001...
/ [4 files][  2.1 GiB/  2.1 GiB]   42.1 MiB/s                                   
==> NOTE: You are performing a sequence of gsutil operations that may
run significantly faster if you instead use gsutil -m cp ... Please
see the -m section under "gsutil help options" for further information
about when gsutil -m can be advantageous.

Copying gs://pegasus_ckpt/model.ckpt-1500000.index...
Copying gs://pegasus_ckpt/model.ckpt-1500000.meta...
Copying gs://pegasus_ckpt/aeslc/model.ckpt-32000.data-00000-of-00001...
Copying gs://pegasus_ckpt/aeslc/model.ckpt-32000.index...
Copying gs://pegasus_ckpt/aeslc/model.ckpt-32000.meta...
Copying gs://pegasus_ckpt/arxiv/model.ckpt-340000.data-00000-of-00001...
Copying gs://pegasus_ckpt/arxiv/model.ckpt-340000.index...
C

In [ ]:
# need this to find pegasus
import os
os.environ['PYTHONPATH'] += ':/content'

As the google pegasus has the pretrained model, I directly jumped to evaluate the model. Unfortunately it doesnot proceed. It throwed up the "ValueError: Could not find trained model in model_dir: ckpt/pegasus_ckpt/aeslc." . And then I Pretained the model. The copy of my pretained model cnn_dailymail is availble in my google drive.If you want to skip the training sesstion as it takes nearly around ~12hrs to complete, you can download it from my drive. https://drive.google.com/drive/folders/1HcWnIXlq3yiUQw0klks25-9T8edM36yP?usp=sharing

In [ ]:
# fine-tune on aeslc
!python3 pegasus/bin/train.py --params=aeslc_transformer \
--param_overrides=vocab_filename=ckpt/pegasus_ckpt/c4.unigram.newline.10pct.96000.model \
--train_init_checkpoint=ckpt/pegasus_ckpt/model.ckpt-1500000 \
--model_dir=ckpt/pegasus_ckpt/cnn_dailymail


Streaming output truncated to the last 5000 lines.
I0725 15:37:47.819032 140580863948672 tpu_estimator.py:2307] global_step/sec: 0.669226
INFO:tensorflow:examples/sec: 5.35381
I0725 15:37:47.819390 140580863948672 tpu_estimator.py:2308] examples/sec: 5.35381
INFO:tensorflow:global_step/sec: 0.66912
I0725 15:37:49.313458 140580863948672 tpu_estimator.py:2307] global_step/sec: 0.66912
INFO:tensorflow:examples/sec: 5.35296
I0725 15:37:49.313697 140580863948672 tpu_estimator.py:2308] examples/sec: 5.35296
INFO:tensorflow:global_step/sec: 0.668084
I0725 15:37:50.810292 140580863948672 tpu_estimator.py:2307] global_step/sec: 0.668084
INFO:tensorflow:examples/sec: 5.34467
I0725 15:37:50.810533 140580863948672 tpu_estimator.py:2308] examples/sec: 5.34467
INFO:tensorflow:global_step/sec: 0.667537
I0725 15:37:52.308315 140580863948672 tpu_estimator.py:2307] global_step/sec: 0.667537
INFO:tensorflow:examples/sec: 5.3403
I0725 15:37:52.308520 140580863948672 tpu_estimator.py:2308] examples/sec: 5.

Evaluation

---

This will start to create your summaries for your input data. Once done you will see 3 text files created in the directory of the model that you pick. These three files correspond to the input text, target text, and the predicted summaries. You can open these text files and analyze the summaries.

In [ ]:
!python3 pegasus/bin/evaluate.py --params=aeslc_transformer \
--param_overrides=vocab_filename=ckpt/pegasus_ckpt/c4.unigram.newline.10pct.96000.model,batch_size=1,beam_size=5,beam_alpha=0.6 \
--model_dir=ckpt/pegasus_ckpt/cnn_dailymail




Instructions for updating:
Use standard file APIs to check for files with this prefix.
W0725 16:42:09.819481 140205032064896 deprecation.py:323] From pegasus/bin/evaluate.py:85: checkpoint_exists (from tensorflow.python.training.checkpoint_management) is deprecated and will be removed in a future version.
Instructions for updating:
Use standard file APIs to check for files with this prefix.

W0725 16:42:10.137942 140205032064896 module_wrapper.py:139] From /content/pegasus/ops/public_parsing_ops.py:93: The name tf.gfile.GFile is deprecated. Please use tf.io.gfile.GFile instead.


W0725 16:42:10.301715 140205032064896 module_wrapper.py:139] From /content/pegasus/params/estimator_utils.py:49: The name tf.ConfigProto is deprecated. Please use tf.compat.v1.ConfigProto instead.

W0725 16:42:10.302333 140205032064896 estimator.py:1994] Estimator's model_fn (<function _estimator_model_fn.<locals>.model_fn at 0x7f83943fae18>) includes params argument, but params are not passed to Estimator.